## Errors 

- not every songs starts with keysignature, time indication
- keysignature should be added (not necessarily in C when it is missing)


In [ ]:
from music21 import *
import copy

In [ ]:
s = converter.parse("/Users/jvo/Downloads/Ultime_Jazz_Book_-_A.mxl")
# s = converter.parse("/Users/jvo/Downloads/Ultime_Jazz_Real_Book_-_A.musicxml")
s = converter.parse("/Users/jvo/Downloads/Ultime_Jazz_Real_Book_-_A(from mscz).mxl")

In [ ]:
# offsetStarts = [keyStart.getOffsetInHierarchy(s) for keyStart in s[key.KeySignature]]
offsetsStartPage = [0] + [el.getOffsetInHierarchy(s) for el in s[layout.PageLayout]]

In [ ]:
len(offsetsStartPage)

In [ ]:
metadataSongs = [[page,None,None] for page in range(1,len(offsetsStartPage) + 1)]

for tb in s[text.TextBox]:
    if tb.page > len(offsetsStartPage):
        continue
    
    if tb.style.justify == 'right':
        metadataSongs[tb.page-1][2] = tb.content
        
    if tb.style.justify == 'center' and not metadataSongs[tb.page-1][1]:
        metadataSongs[tb.page-1][1] = tb.content

metadataSongs

In [ ]:
noStartPages = []

for datum in metadataSongs:
    if not datum[2]:
        noStartPages.append(datum[0])

offsetsStartSong = [offset for i, offset in enumerate(offsetsStartPage) if i not in noStartPages]
offsetsStartSong

In [ ]:
measureNumbersStartSong = []
measures = s[stream.Measure]
j = 1
for i in range(len(measures)):
    if measures[i].offset in offsetsStartSong:
        measureNumbersStartSong.append(measures[i].number)

#         indexPrevMeasure = max(0,i-1)
#         barlinePrevMeasure = measures[indexPrevMeasure][bar.Barline].last()
#         if barlinePrevMeasure and barlinePrevMeasure.type=='final' or True:
#             measureNumbersStart.append(measures[i].number)

lastMeasureNumber = s[stream.Measure][-1].number
measureNumbersStartEndSong = []

for measureNumberStart, measureNumberStartNext in zip(measureNumbersStartSong, measureNumbersStartSong[1:]):
    measureNumbersStartEndSong.append((measureNumberStart, measureNumberStartNext))

measureNumbersStartEndSong.append((measureNumbersStartSong[-1], lastMeasureNumber))

len(measureNumbersStartEndSong)

In [ ]:
songs = []
for measureNumberStart, measureNumberEnd in measureNumbersStartEndSong:
    songs.append(s.measures(measureNumberStart, measureNumberEnd - 1))

    

In [ ]:
for i, song in enumerate(songs):
    m = song[stream.Measure][0]
    if m[layout.PageLayout]:
        
        pl = m[layout.PageLayout][0]
        m.remove(pl)
        m.informSites()
        
    song.insert(0, metadata.Metadata())
    song.metadata.title = metadataSongs[i][1]
    song.metadata.composer = metadataSongs[i][2]


In [ ]:
a = songs[1][harmony.ChordSymbol][1]
c = songs[1][harmony.ChordSymbol][2]
a.style.absoluteY

In [ ]:
songs[1].show()
